In [ ]:
def shuffle_weights(model, weights=None):
    if weights is None:
        weights = model.get_weights()
    weights = [np.random.permutation(w.flat).reshape(w.shape) for w in weights]
    model.set_weights(weights)

In [ ]:
shuffle_weights(model)

# MOBILE NET

In [ ]:
!rm -rf data
!unzip data.zip > /dev/null
!rm -rf __MACOSX/

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
import warnings


In [ ]:
os.chdir("data")
if os.path.isdir("train") is False:
  os.mkdir("train")
  os.mkdir("valid")
  os.mkdir("test")

  shutil.move(f'bottom', 'train')
  shutil.move(f'front', 'train')
  shutil.move(f'left', 'train')
  shutil.move(f'right', 'train')
  shutil.move(f'top', 'train')

  os.mkdir(f'valid/bottom')
  os.mkdir(f'valid/front')
  os.mkdir(f'valid/left')
  os.mkdir(f'valid/right')
  os.mkdir(f'valid/top')

  os.mkdir(f'test/bottom')
  os.mkdir(f'test/front')
  os.mkdir(f'test/left')
  os.mkdir(f'test/right')
  os.mkdir(f'test/top')

  bottom_valid_samples = random.sample(os.listdir(f'train/bottom'),50)
  front_valid_samples = random.sample(os.listdir(f'train/front'),50)
  left_valid_samples = random.sample(os.listdir(f'train/left'),50)
  right_valid_samples = random.sample(os.listdir(f'train/right'),50)
  top_valid_samples = random.sample(os.listdir(f'train/top'),50)

  for img in bottom_valid_samples:
    shutil.move(f'train/bottom/{img}', f'valid/bottom')

  for img in front_valid_samples:
    shutil.move(f'train/front/{img}', f'valid/front')

  for img in left_valid_samples:
    shutil.move(f'train/left/{img}', f'valid/left')

  for img in right_valid_samples:
    shutil.move(f'train/right/{img}', f'valid/right')

  for img in top_valid_samples:
    shutil.move(f'train/top/{img}', f'valid/top')

  bottom_test_samples = random.sample(os.listdir(f'train/bottom'),100)
  front_test_samples = random.sample(os.listdir(f'train/front'),100)
  left_test_samples = random.sample(os.listdir(f'train/left'),100)
  right_test_samples = random.sample(os.listdir(f'train/right'),100)
  top_test_samples = random.sample(os.listdir(f'train/top'),100)

  for img in bottom_test_samples:
    shutil.move(f'train/bottom/{img}', f'test/bottom')

  for img in front_test_samples:
    shutil.move(f'train/front/{img}', f'test/front')

  for img in left_test_samples:
    shutil.move(f'train/left/{img}', f'test/left')

  for img in right_test_samples:
    shutil.move(f'train/right/{img}', f'test/right')

  for img in top_test_samples:
    shutil.move(f'train/top/{img}', f'test/top')

os.chdir("..")

In [ ]:
train_path = "data/train"
valid_path = "data/valid"
test_path = "data/test"

# train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)\
train_batches = ImageDataGenerator()\
  .flow_from_directory(directory=train_path, target_size=(224,224), batch_size=10)
# valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)\
valid_batches = ImageDataGenerator()\
  .flow_from_directory(directory=valid_path, target_size=(224,224), batch_size=10)
# test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)\
test_batches = ImageDataGenerator()\
  .flow_from_directory(directory=test_path, target_size=(224,224), batch_size=10, shuffle=False)

Found 16289 images belonging to 5 classes.
Found 250 images belonging to 5 classes.
Found 500 images belonging to 5 classes.


In [ ]:
mobile_model = tf.keras.applications.mobilenet.MobileNet()

x = mobile_model.layers[-5].output
output = Flatten()(x)
output = Dense(units=5, activation='softmax')(output)

model = Model(inputs=mobile_model.input, outputs=output)

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x=train_batches, validation_data=valid_batches, epochs=4, steps_per_epoch=1620, verbose=2)

Epoch 1/4
1620/1620 - 90s - loss: 0.0761 - accuracy: 0.9744 - val_loss: 0.0169 - val_accuracy: 0.9960 - 90s/epoch - 56ms/step
Epoch 2/4
1620/1620 - 64s - loss: 0.0139 - accuracy: 0.9961 - val_loss: 0.0354 - val_accuracy: 0.9920 - 64s/epoch - 39ms/step
Epoch 3/4
1620/1620 - 65s - loss: 0.0111 - accuracy: 0.9967 - val_loss: 0.0322 - val_accuracy: 0.9880 - 65s/epoch - 40ms/step
Epoch 4/4
1620/1620 - 64s - loss: 0.0119 - accuracy: 0.9961 - val_loss: 0.0119 - val_accuracy: 0.9920 - 64s/epoch - 39ms/step


In [ ]:
model.save("models/eyes_model.h5")

In [ ]:
predictions = model.predict(x=test_batches, steps=100, verbose=0)

In [ ]:
np.argmax(predictions, axis=-1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3,

In [ ]:
test_batches.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3,

In [ ]:
error = np.absolute(test_batches.classes - np.argmax(predictions, axis=-1)).sum() / len(test_batches.classes)
print("accuracy is:", (1 - error) * 100, "%")

accuracy is: 100.0 %
